- 데이터가 복잡하고 양이 많다면 클라우드환경 (ex 구글 코랩, 드라이브)에 업로드하는 시간이 매우 오래걸림
- 로컬 환경인 쥬피터 노트북으로 빠르게 전처리를 하고 그 결과를 NPZ(numpy zip)파일로 저장
- 저장된 NPZ 파일을 원하는 환경으로 불러와서 사용하자!

In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.model_selection import train_test_split
# os: 파일, 폴더 처리에 관련된 라이브러리
import os

In [10]:
# cats_and_dogs_filtered 폴더 내에 각각 데이터셋들의 경로를 설정
train_cats_dir = './data/cats_and_dogs_filtered/train/cats/'
train_dogs_dir = './data/cats_and_dogs_filtered/train/dogs/'
test_cats_dir = './data/cats_and_dogs_filtered/test/cats/'
test_dogs_dir = './data/cats_and_dogs_filtered/test/dogs/'

In [11]:
# os.listdir() : 해당 경로에 있는 파일명들을 리스트에 순서대로 저장
train_cats_fnames = os.listdir(train_cats_dir)
train_dogs_fnames = os.listdir(train_dogs_dir)
test_cats_fnames = os.listdir(test_cats_dir)
test_dogs_fnames = os.listdir(test_dogs_dir)

In [13]:
# os.path.join : 파일 경로와 파일명을 결합
test_path = os.path.join(train_cats_dir, train_cats_fnames[0])
test_path

'./data/cats_and_dogs_filtered/train/cats/cat.0.jpg'

In [21]:
# 사진을 로딩하는 함수를 만들어보자!
# 사진 크기는 전부 224 x 224로 고정시키자
# 신경망에 넣기위해서는 데이터의 크기를 같게 만들어줘야 함

def load_images(folder_path, file_names, img_size_shape=(224,224)) :
    images = []
    
    for i in file_names :
        # 폴더 경로 + 파일명 합치기
        path = os.path.join(folder_path, i)
        # 파일 오픈 및 크기조정(resize : 파일의 형태(사이즈)를 변형시켜줌)
        img = Image.open(path).resize(img_size_shape)
        # numpy 배열로 변경 후 리스트에 추가
        images.append(np.array(img))
        
    # 리스트도 numpy 배열로 변환하여 리턴
    return np.array(images)

In [22]:
X_train_cats = load_images (train_cats_dir, train_cats_fnames)
X_train_dogs = load_images (train_dogs_dir, train_dogs_fnames)
X_test_cats = load_images (test_cats_dir, test_cats_fnames)
X_test_dogs = load_images (test_dogs_dir, test_dogs_fnames)

In [24]:
print(X_train_cats.shape)
print(X_train_dogs.shape)
print(X_test_cats.shape)
print(X_test_dogs.shape)

(1000, 224, 224, 3)
(1000, 224, 224, 3)
(500, 224, 224, 3)
(500, 224, 224, 3)


In [25]:
# 데이터 병합 (고양이, 개 이미지 배열 데이터를 순서대로 합쳐줌)
X_train = np.concatenate((X_train_cats, X_train_dogs))
X_test = np.concatenate((X_test_cats,X_test_dogs))

In [27]:
X_train.shape, X_test.shape

((2000, 224, 224, 3), (1000, 224, 224, 3))

In [29]:
# 정답 데이터 만들기 (고양이 : 0, 개 : 1)
y_train = np.array([0]*1000 + [1]*1000)
y_test = np.array([0]*500 + [1]*500)

In [30]:
y_train

array([0, 0, 0, ..., 1, 1, 1])

In [31]:
# 0~255 사이의 픽셀 값을 0~1사이로 줄여줄 수 있음
# 수치값이 작아지기 때문에 연산속도를 향상시킬 수 있음
# 데이터의 픽셀 수치값들을 실수로 변경 시킨 후에 나눠줌
X_train = X_train.astype("float32") /255
X_test = X_test.astype ("float32") /255

In [32]:
# NPZ 파일로 저장
np.savez_compressed ('./data/cats_and_dogs.npz', # 폴더 경로와 파일명을 설정
                    X_train = X_train,           # 학습용 문제 데이터 설정
                    X_test = X_test,             # 평가용 문제 데이터 설정
                    y_train = y_train,           # 학습용 정답 데이터 설정
                    y_test = y_test              # 평가용 정답 데이터 설정
                    )